In [1]:
import pandas
import sklearn
import numpy as np

In [2]:
train = pandas.read_csv("linear_train.txt", names=["word", "y"])
test = pandas.read_csv("linear_test.txt", names=["word"])
ans = pandas.read_csv("linear_ans_example.txt")

In [3]:
from sklearn.linear_model import LogisticRegression

In [4]:
from sklearn.model_selection import cross_val_score
from collections import defaultdict

In [94]:
class ExtractFeatures(sklearn.base.BaseEstimator, sklearn.base.TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return [self.extract(word) for word in X]

    def extract(self, word):
        features = defaultdict(lambda: 0)
        word = unicode(word, "utf-8")
        if word[0].isupper() and not word.isupper():
            features['capitalized'] = 1
        if word.isupper():
            features['allcaps'] = 1
        word = word.lower()
        for i in range(min(len(word), 7)):
            features["pref" + word[:i]] += 1
            features["suf" + word[-i:]] += 1
            if i > 0:
                features["word" + word[:-i] + "*"*i] += 1
        for length in range(1, 6):
            for i in range(len(word) - length + 1):
                features[word[i:i + length]] += 1
            for offset in range(1, 4):
                if len(word) - offset - length < 0:
                    continue
                features[word[len(word) - offset - length: len(word) - offset] + "*"*offset] += 1
        features["length"] = len(word)
        features["rect_len"] = max(len(word) - 10, 0)
        new_features = defaultdict(lambda: 0)
        for key1 in features.iterkeys():
            for key2 in features.iterkeys():
                new_features[u"{}|{}".format(key1, key2)] += 1
        return features.update(new_features)
        

In [95]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import StratifiedKFold

In [96]:
vectorizer = DictVectorizer()
train_data = vectorizer.fit_transform(ExtractFeatures().transform(train.word))

AttributeError: 'NoneType' object has no attribute 'iteritems'

In [ ]:
test_data = vectorizer.transform(ExtractFeatures().transform(test.word))

In [77]:
#est = sklearn.linear_model.SGDClassifier(loss='log', penalty="l2", alpha=0.0001, n_iter=20)
est = sklearn.linear_model.LogisticRegression(C=1)
#est = sklearn.linear_model.LogisticRegressionCV(30, scoring="roc_auc", solver="liblinear")
est = make_pipeline(
                    sklearn.preprocessing.MaxAbsScaler(),
                    est)

In [78]:
#cv = sklearn.model_selection.GridSearchCV(est, {"logisticregression__C": np.exp(np.linspace(-4, 4, 10))}, scoring="roc_auc")
cross_val_score(est, train_data, train.y, scoring="roc_auc", cv=StratifiedKFold(shuffle=True)).mean()

0.91301564313078742

In [64]:
cv.fit(train_data, train.y)
answers = cv.predict_proba(test_data)[:, 1]

In [20]:
ans.Answer = answers

In [21]:
ans.to_csv("answer.txt", index=False)

In [62]:
est.get_params().keys()

['logisticregression__n_jobs',
 'logisticregression__multi_class',
 'logisticregression',
 'logisticregression__penalty',
 'logisticregression__C',
 'logisticregression__random_state',
 'logisticregression__max_iter',
 'maxabsscaler',
 'logisticregression__warm_start',
 'logisticregression__verbose',
 'logisticregression__class_weight',
 'steps',
 'logisticregression__dual',
 'maxabsscaler__copy',
 'logisticregression__tol',
 'logisticregression__intercept_scaling',
 'logisticregression__solver',
 'logisticregression__fit_intercept']

In [70]:
cv.best_score_

0.82268825474931884